Kernel Ridge regression using Polynomial and RBF kernels.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def generate_data(n):
    # np.random.seed(100)
    x_ = np.random.uniform(0, 1, size=(n,))
    y_ = f_star(x_) + np.random.normal(size=(n,))
    return x_, y_


def f_star(z):
    return 4 * np.sin(np.pi * z) * np.cos(6 * np.pi * (z ** 2))


def polyKernel():
    def function(z, x, d):
        return np.power((1 + x * z), d)

    return function


def kRbf():
    def function(x, z, gamma):
        return np.exp(-gamma * np.power(x - z, 2))

    return function


# returns the F"hat" values for a fine x (for plots)
# fine x : line space between 0 and 1 of 1000 points
def getF_Hat(x_, y_, kernel_f, lambda_, hyper, n_):
    def function():
        def f(z1, z2):
            return kernel_f(z1, z2, hyper)

        return f

    x_f = np.linspace(0, 1, 1000)
    f_ = kernel_ridge(x_, y_, function(), lambda_, n_)
    y_pred = [f_(x_i) for x_i in x_f]
    return x_f, y_pred


def kernel_ridge(x_, y_, kernel_f, lam, n_):
    K = np.fromfunction(lambda i, j: kernel_f(x_[i], x_[j]), dtype=int, shape=(n_, n_))
    alpha = np.linalg.solve(K + lam * np.eye(n_), y_)

    def f(z):
        return np.sum(np.dot(alpha, kernel_f(z, x_)))

    return f


def LOO(x_, y_, kernel_f, lambdas, hypers, n_):
    errors = np.empty((len(lambdas), len(hypers)))
    for i, lambda_ in enumerate(lambdas):
        for j, hyper in enumerate(hypers):
            error = 0
            for k in range(n_):
                batchX = np.concatenate((x_[:k], x_[k + 1:]))
                batchY = np.concatenate((y_[:k], y_[k + 1:]))

                def kernelFunc(z1, z2):
                    return kernel_f(z1, z2, hyper)

                error += (y_[k] - kernel_ridge(batchX, batchY, kernelFunc, lambda_, n_ - 1)(x_[k])) ** 2

            errors[i, j] = error / n_

    return get_Lambda_Hyper(errors, lambdas, hypers)


def CV10(x_, y_, kernel_f, lambdas, hypers, n_):
    errors = np.empty((len(lambdas), len(hypers)))
    for i, lambda_ in enumerate(lambdas):
        for j, hyper in enumerate(hypers):
            error = 0
            for k in range(10):
                i1 = (n_ // 10) * k
                i2 = (n_ // 10) * (k + 1)
                batchX = np.concatenate((x_[:i1], x_[i2:]))
                batchY = np.concatenate((y_[:i1], y_[i2:]))

                def kernelFunc(z1, z2):
                    return kernel_f(z1, z2, hyper)

                asArray = np.asarray([kernel_ridge(batchX, batchY, kernelFunc, lambda_, (n - n // 10))(z)
                                      for z in x_[i1:i2]])
                error += np.sum(np.square((y_[i1:i2] - asArray))) / len(x_[i1:i2])

            errors[i, j] = error

    return get_Lambda_Hyper(errors, lambdas, hypers)


# returns the lambda and hyper parameter in argmin of errors
def get_Lambda_Hyper(errors_, lambdas, hypers):
    indexLambda = np.unravel_index(np.argmin(errors_, axis=None), errors_.shape)[0]
    indexHyper = np.unravel_index(np.argmin(errors_, axis=None), errors_.shape)[1]
    return lambdas[indexLambda], hypers[indexHyper]


def A3_b(x_, y_, kernel_f, lambda_, hyper, n_, kernelID):
    if kernelID == 0:
        name = "Poly"
    else:
        name = "RBF"

    x_f, y_pred = getF_Hat(x_, y_, kernel_f, lambda_, hyper, n_)

    plt.plot(x_, y_, 'ko')
    plt.plot(x_f, f_star(x_f))
    plt.plot(x_f, y_pred)
    # plt.scatter(x_, y_)
    plt.legend(['Data', 'Actual', 'Kernel Ridge Regression'])
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Kernel Ridge Regression: n = ' + str(n_) + ' -- ' + name + ' - A3.b')
    plt.show()


def bootStrap(x_, y_, kernel_f, lambda_, hyper, n_, B_, kernelID):
    if kernelID == 0:
        name = "Poly"
    else:
        name = "RBF"

    f_p = np.zeros((B_, 1000))

    def kernel_function(z1, z2):
        return kernel_f(z1, z2, hyper)

    for b_ in range(B_):
        batchIndex = np.random.choice(np.arange(n_), n_, replace=True)  # replacement = true
        batchX = x_[batchIndex]
        batchY = y_[batchIndex]
        f_hat = kernel_ridge(batchX, batchY, kernel_function, lambda_, n_)
        x_f = np.linspace(0, 1, 1000)
        y_p = []
        for x_i in x_f:
            y_p.append(f_hat(x_i))
        f_p[b_, :] = y_p

    p_5 = np.percentile(f_p, 5, axis=0)
    p_95 = np.percentile(f_p, 95, axis=0)

    f_hat = kernel_ridge(x_, y_, kernel_function, lambda_, n_)
    x_f = np.linspace(0, 1, 1000)
    y_p = []
    for x_i in x_f:
        y_p.append(f_hat(x_i))

    plt.plot(x_, y_, 'ko')
    plt.plot(x_f, f_star(x_f))
    plt.plot(x_f, y_p)
    plt.plot(x_f, p_5, 'r-')
    plt.plot(x_f, p_95, 'b-')
    plt.legend(['Data', 'Actual', 'F_hat', '5th Percentile', '95th Percentile'])
    plt.title('BootSrap : n = ' + str(n_) + " -- Kernel : " + name)
    plt.show()


# ===================================== A.3: a, b, c ===================================== #

n = 30
print("================================")
print("n = " + str(n))

X, Y = generate_data(n)

params_d = np.arange(1, 50)
params_lambda = np.power(10 * np.ones(6), np.arange(-6, 0))

# poly kernel
best_lambda, best_d = LOO(X, Y, polyKernel(), params_lambda, params_d, n)

print("K-poly ----> best lambda: ", best_lambda, " best d: ", best_d)

A3_b(X, Y, polyKernel(), best_lambda, best_d, n, 0)
B = 300
bootStrap(X, Y, polyKernel(), best_lambda, best_d, n, B, 0)

# RBF kernel

# get the heuristic gamma ball park
# estimateGamma = 1 / np.median(np.square([(X[i] - X[j]) for i in range(n) for j in range(n)]))
# print('gamma heuristic estimate: ', estimateGamma)

params_gamma = np.linspace(0, 30, 20)

params_lambda = np.power(10 * np.ones(6), np.arange(-6, 0))
best_lambda, best_gamma = LOO(X, Y, kRbf(), params_lambda, params_gamma, n)

print("K-RBF ----> best lambda: ", best_lambda, " best Gamma: ", best_gamma)

A3_b(X, Y, polyKernel(), best_lambda, best_d, n, 1)
B = 300
bootStrap(X, Y, polyKernel(), best_lambda, best_d, n, B, 1)

# ===================================== A.3: d ===================================== #

n = 300
print("================================")
print("n = " + str(n))

X, Y = generate_data(n)

params_d = np.arange(1, 20)
params_lambda = np.power(10 * np.ones(6), np.arange(-6, 0))

# poly kernel
best_lambda_poly_300, best_d = CV10(X, Y, polyKernel(), params_lambda, params_d, n)

print("K-poly ----> best lambda: ", best_lambda_poly_300, " best d: ", best_d)

A3_b(X, Y, polyKernel(), best_lambda_poly_300, best_d, n, 0)
B = 300
bootStrap(X, Y, polyKernel(), best_lambda_poly_300, best_d, n, B, 0)

# RBF kernel

# get the heuristic gamma ball park
# estimateGamma = 1 / np.median(np.square([(X[i] - X[j]) for i in range(n) for j in range(n)]))
# print('gamma heuristic estimate: ', estimateGamma)

params_gamma = np.linspace(0, 30, 20)

params_lambda = np.power(10 * np.ones(6), np.arange(-6, 0))

best_lambda_rbf_300, best_gamma = CV10(X, Y, kRbf(), params_lambda, params_gamma, n)

print("K-RBF ----> best lambda: ", best_lambda_rbf_300, " best Gamma: ", best_gamma)

A3_b(X, Y, polyKernel(), best_lambda_rbf_300, best_d, n, 1)
B = 300
bootStrap(X, Y, polyKernel(), best_lambda_rbf_300, best_d, n, B, 1)

# ===================================== A.3: e ===================================== #

m = 1000

X_m, Y_m = generate_data(m)


def kernel_function_poly():
    def getF(z1, z2):
        return polyKernel()(z1, z2, best_d)

    return getF


def kernel_function_rbf():
    def getF(z1, z2):
        return kRbf()(z1, z2, best_gamma)

    return getF


f_p = kernel_ridge(X, Y, kernel_function_poly(), best_lambda_poly_300, n)
f_r = kernel_ridge(X, Y, kernel_function_rbf(), best_lambda_rbf_300, n)

B = 300
values = np.empty(B)

for b in range(B):
    batchIndex = np.random.choice(np.arange(m), m, replace=True)
    batchX = X_m[batchIndex]
    batchY = Y_m[batchIndex]

    polyArray = np.asarray([f_p(x_i) for x_i in batchX])
    rbfArray = np.asarray([f_r(x_i) for x_i in batchX])

    # squares of difference
    polyErr = np.square(batchY - polyArray)
    rbfErr = np.square(batchY - rbfArray)

    values[b] = np.mean(polyErr - rbfErr)

p_5 = np.percentile(values, 5)
p_95 = np.percentile(values, 95)

print("================================")
print("Confidence Interval:")
print("[" + str(p_5) + " , " + str(p_95) + "]")